## SQL at Scale with Spark SQL

Welcome to the SQL mini project. For this project, you will use the Databricks Platform and work through a series of exercises using Spark SQL. The dataset size may not be too big but the intent here is to familiarize yourself with the Spark SQL interface which scales easily to huge datasets, without you having to worry about changing your SQL queries. 

The data you need is present in the mini-project folder in the form of three CSV files. This data will be imported in Databricks to create the following tables under the __`country_club`__ database.

<br>
1. The __`bookings`__ table,
2. The __`facilities`__ table, and
3. The __`members`__ table.

You will be uploading these datasets shortly into the Databricks platform to understand how to create a database within minutes! Once the database and the tables are populated, you will be focusing on the mini-project questions.

In the mini project, you'll be asked a series of questions. You can solve them using the databricks platform, but for the final deliverable,
please download this notebook as an IPython notebook (__`File -> Export -> IPython Notebook`__) and upload it to your GitHub.

### Creating the Database

We will first create our database in which we will be creating our three tables of interest

In [0]:
%sql 
drop database if exists country_club cascade;
create database country_club;
show databases;

databaseName
country_club
default


### Creating the Tables

In this section, we will be creating the three tables of interest and populate them with the data from the CSV files already available to you. 
To get started, first upload the three CSV files to the DBFS as depicted in the following figure

![](https://i.imgur.com/QcCruBr.png)


Once you have done this, please remember to execute the following code to build the dataframes which will be saved as tables in our database

In [0]:
# File location and type
file_location_bookings = "/FileStore/tables/Bookings.csv"
file_location_facilities = "/FileStore/tables/Facilities.csv"
file_location_members = "/FileStore/tables/Members.csv"

file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
bookings_df = (spark.read.format(file_type) 
                    .option("inferSchema", infer_schema) 
                    .option("header", first_row_is_header) 
                    .option("sep", delimiter) 
                    .load(file_location_bookings))

facilities_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_facilities))

members_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_members))

### Viewing the dataframe schemas

We can take a look at the schemas of our potential tables to be written to our database soon

In [0]:
print('Bookings Schema')
bookings_df.printSchema()
print('Facilities Schema')
facilities_df.printSchema()
print('Members Schema')
members_df.printSchema()

Bookings Schema
root
-- bookid: integer (nullable = true)
-- facid: integer (nullable = true)
-- memid: integer (nullable = true)
-- starttime: timestamp (nullable = true)
-- slots: integer (nullable = true)

Facilities Schema
root
-- facid: integer (nullable = true)
-- name: string (nullable = true)
-- membercost: double (nullable = true)
-- guestcost: double (nullable = true)
-- initialoutlay: integer (nullable = true)
-- monthlymaintenance: integer (nullable = true)

Members Schema
root
-- memid: integer (nullable = true)
-- surname: string (nullable = true)
-- firstname: string (nullable = true)
-- address: string (nullable = true)
-- zipcode: integer (nullable = true)
-- telephone: string (nullable = true)
-- recommendedby: integer (nullable = true)
-- joindate: timestamp (nullable = true)

### Create permanent tables
We will be creating three permanent tables here in our __`country_club`__ database as we discussed previously with the following code

In [0]:
permanent_table_name_bookings = "country_club.Bookings"
bookings_df.write.format("parquet").saveAsTable(permanent_table_name_bookings)

permanent_table_name_facilities = "country_club.Facilities"
facilities_df.write.format("parquet").saveAsTable(permanent_table_name_facilities)

permanent_table_name_members = "country_club.Members"
members_df.write.format("parquet").saveAsTable(permanent_table_name_members)

### Refresh tables and check them

In [0]:
%sql
use country_club;
REFRESH table bookings;
REFRESH table facilities;
REFRESH table members;
show tables;

database,tableName,isTemporary
country_club,bookings,false
country_club,facilities,false
country_club,members,false


### Test a sample SQL query

__Note:__ You can use __`%sql`__ at the beginning of a cell and write SQL queries directly as seen in the following cell. Neat isn't it!

In [0]:
%sql
select * from bookings limit 3

bookid,facid,memid,starttime,slots
0,3,1,2012-07-03T11:00:00.000+0000,2
1,4,1,2012-07-03T08:00:00.000+0000,2
2,6,0,2012-07-03T18:00:00.000+0000,2


#### Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

In [0]:
%sql
select name from facilities where membercost > 0

name
Tennis Court 1
Tennis Court 2
Massage Room 1
Massage Room 2
Squash Court


####  Q2: How many facilities do not charge a fee to members?

In [0]:
%sql
select count(*) from facilities where membercost = 0

count(1)
4


#### Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? 
#### Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [0]:
%sql
select facid, name, membercost, monthlymaintenance from facilities where membercost < 0.2 * monthlymaintenance

facid,name,membercost,monthlymaintenance
0,Tennis Court 1,5.0,200
1,Tennis Court 2,5.0,200
2,Badminton Court,0.0,50
3,Table Tennis,0.0,10
4,Massage Room 1,9.9,3000
5,Massage Room 2,9.9,3000
6,Squash Court,3.5,80
7,Snooker Table,0.0,15
8,Pool Table,0.0,15


#### Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [0]:
%sql
select * from facilities where facid = 1 
union 
select * from facilities where facid = 5

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
1,Tennis Court 2,5.0,25.0,8000,200
5,Massage Room 2,9.9,80.0,4000,3000


#### Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100? 
#### Return the name and monthly maintenance of the facilities in question.

In [0]:
%sql
select case when monthlymaintenance > 100 then 'expensive' else 'cheap' end as label, name, monthlymaintenance from facilities 

label,name,monthlymaintenance
expensive,Tennis Court 1,200
expensive,Tennis Court 2,200
cheap,Badminton Court,50
cheap,Table Tennis,10
expensive,Massage Room 1,3000
expensive,Massage Room 2,3000
cheap,Squash Court,80
cheap,Snooker Table,15
cheap,Pool Table,15


#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.

In [0]:
%sql
select concat(last(surname), ",", last(firstname)) as lastMember from members 

lastMember
"Smith,Darren"


####  Q7: How can you produce a list of all members who have used a tennis court?
- Include in your output the name of the court, and the name of the member formatted as a single column. 
- Ensure no duplicate data
- Also order by the member name.

In [0]:
%sql
select distinct concat(surname, firstname) as memberName, name as courtName
from bookings 
join facilities on bookings.facid = facilities.facid 
join members on bookings.memid = members.memid 
order by memberName

memberName,courtName
BaderFlorence,Tennis Court 2
BaderFlorence,Pool Table
BaderFlorence,Tennis Court 1
BaderFlorence,Squash Court
BaderFlorence,Massage Room 2
BaderFlorence,Table Tennis
BaderFlorence,Badminton Court
BaderFlorence,Snooker Table
BakerAnne,Tennis Court 1
BakerAnne,Tennis Court 2


#### Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? 

- Remember that guests have different costs to members (the listed costs are per half-hour 'slot')
- The guest user's ID is always 0. 

#### Include in your output the name of the facility, the name of the member formatted as a single column, and the cost.

- Order by descending cost, and do not use any subqueries.

In [0]:
%sql
select name as facilityName,
  concat(surname, ",", firstname) as memberName,
  case when members.memid = 0 then guestcost * slots
  else membercost * slots end as cost
from bookings
join facilities on bookings.facid = facilities.facid 
join members on bookings.memid = members.memid
where date(starttime) = "2012-09-14"
and 
((members.memid = 0 and guestcost * slots >= 30) or 
(members.memid != 0 and membercost * slots >= 30))
order by cost desc

facilityName,memberName,cost
Massage Room 2,"GUEST,GUEST",320.0
Massage Room 1,"GUEST,GUEST",160.0
Massage Room 1,"GUEST,GUEST",160.0
Massage Room 1,"GUEST,GUEST",160.0
Tennis Court 2,"GUEST,GUEST",150.0
Tennis Court 2,"GUEST,GUEST",75.0
Tennis Court 1,"GUEST,GUEST",75.0
Tennis Court 1,"GUEST,GUEST",75.0
Squash Court,"GUEST,GUEST",70.0
Massage Room 1,"Farrell,Jemima",39.6


#### Q9: This time, produce the same result as in Q8, but using a subquery.

In [0]:
%sql
select name as facilityName,
  concat(surname, ",", firstname) as memberName,
  cost
from (
select *,
  case when members.memid = 0 then guestcost * slots
  else membercost * slots end as cost
from bookings
join facilities on bookings.facid = facilities.facid 
join members on bookings.memid = members.memid
where date(starttime) = "2012-09-14"
)
where cost >= 30
order by cost desc

facilityName,memberName,cost
Massage Room 2,"GUEST,GUEST",320.0
Massage Room 1,"GUEST,GUEST",160.0
Massage Room 1,"GUEST,GUEST",160.0
Massage Room 1,"GUEST,GUEST",160.0
Tennis Court 2,"GUEST,GUEST",150.0
Tennis Court 2,"GUEST,GUEST",75.0
Tennis Court 1,"GUEST,GUEST",75.0
Tennis Court 1,"GUEST,GUEST",75.0
Squash Court,"GUEST,GUEST",70.0
Massage Room 1,"Farrell,Jemima",39.6


#### Q10: Produce a list of facilities with a total revenue less than 1000.
- The output should have facility name and total revenue, sorted by revenue. 
- Remember that there's a different cost for guests and members!

In [0]:
%sql
select *
from
(
select name as facilityName,
sum(income) - first(initialoutlay) - 12 * first(monthlymaintenance) as Revenue
from
(
select name, 
  initialoutlay,
  monthlymaintenance,
  case when members.memid = 0 then guestcost * slots
  else membercost * slots end as income
from bookings
join facilities on bookings.facid = facilities.facid 
join members on bookings.memid = members.memid
) fullTable
group by name
) groupedTable
where Revenue < 1000
order by Revenue desc

facilityName,Revenue
Table Tennis,-260.0
Pool Table,-310.0
Snooker Table,-390.0
Badminton Court,-2693.5
Massage Room 2,-25545.400000000005
